In [17]:
import pandas as pd
import numpy as np
import glob
from openpyxl import load_workbook
import sys

In [19]:

file_path = 'C:/Users/moonj/Desktop/남동발전표준화/자동화/upload_terms/asset/표준화 작업-71.계약이행실적증명발급_V0.97.xlsx'

wb = load_workbook(filename=file_path, data_only=True)
sheetname = wb.sheetnames[1]

df = pd.read_excel(file_path, sheet_name=sheetname)
print(f'read_excel_file : {file_path} | Sheet name : {sheetname}')
tmp_df = df.rename(columns=df.iloc[0])
extract_df = tmp_df.drop(tmp_df.index[0]) 
extract_df.fillna('', inplace=True)
extract_df=extract_df[(extract_df['미등록용어']!='')&(extract_df['분류어chk']!='CD')&(extract_df['미사용구분']=='')&(extract_df['분류어chk']!='')]        
target_df = extract_df[['한글용어명','영문용어명','분류어chk','데이터 타입','데이터 길이']]
target_df

read_excel_file : C:/Users/moonj/Desktop/남동발전표준화/자동화/upload_terms/asset/표준화 작업-71.계약이행실적증명발급_V0.97.xlsx | Sheet name : ★메타데이터


,한글용어명,영문용어명,분류어chk,데이터 타입,데이터 길이
217,사업소팩스번호,BIZP_FXNO,FXNO,VARCHAR2,15
218,지역전화번호,AREA_TELNO,TELNO,VARCHAR2,4
221,구매담당여부,BUY_TCHG_YN,YN,CHAR,1
222,공사담당여부,CONS_TCHG_YN,YN,CHAR,1
223,용역담당여부,SVL_TCHG_YN,YN,CHAR,1
224,간이공사담당여부,SMPLC_CONS_TCHG_YN,YN,CHAR,1
225,하도급담당여부,SBCN_TCHG_YN,YN,CHAR,1
228,최상위관리자여부,UPPOS_MNGR_YN,YN,CHAR,1
231,상태코드값,STS_CD_VAL,VAL,CHAR,1
238,리스트코드명,LIST_CD_NM,NM,VARCHAR2,10


In [43]:
test_df = target_df.groupby(['분류어chk','데이터 타입','데이터 길이'])['한글용어명'].count().reset_index()
test_df.rename(columns={'한글용어명':'빈도수'}, inplace=True)

set_df = test_df.groupby(['분류어chk','데이터 타입','데이터 길이'])['빈도수'].nunique().reset_index()

set_df = set_df[(set_df['분류어chk']=='NM')&(set_df['데이터 타입']=='VARCHAR2')]
len(set_df['빈도수'].unique())


1

In [38]:
test_df

,분류어chk,데이터 타입,데이터 길이,빈도수
0,FXNO,VARCHAR2,15,1
1,NM,VARCHAR2,10,1
2,NM,VARCHAR2,20,1
3,TELNO,VARCHAR2,4,1
4,VAL,CHAR,1,1
5,YN,CHAR,1,7


In [23]:
target_df = target_df.astype({'한글용어명':'string',
                             '영문용어명':'string',
                             '분류어chk':'string',
                             '데이터 타입':'string',
                             '데이터 길이':'float'})

target_df.dtypes

한글용어명      string
영문용어명      string
분류어chk     string
데이터 타입     string
데이터 길이    float64
dtype: object

In [5]:
domain_file = '/Users/jihyun/Desktop/upload_terms/domain.xlsx'

domain_df = pd.read_excel(domain_file)

domain_df = domain_df.astype({'약어':'string',
                            '단어명':'string',
                            '공통표준도메인명':'string',
                            '데이터타입':'string',
                            '데이터길이':'string'})

domain_df['데이터길이']=domain_df['데이터길이'].apply(lambda x : x.replace(',','.'))

domain_df = domain_df.astype({'데이터길이':'float'})

domain_df.dtypes

약어           string
단어명          string
공통표준도메인명     string
데이터타입        string
데이터길이       float64
데이터길이1        int64
데이터소수점길이     object
비고           object
dtype: object

In [23]:
def find_nearlist_val(data_length_list, data_length):
    new_list = [x for x in data_length_list if x>=data_length]
    if len(new_list) == 0 :
        return '' 
    
    else : 
        minValue = min(new_list, key = lambda x : abs(x-data_length))
        return minValue



def select_domin(domain_df, target_last_word, data_type, data_length):

    if target_last_word in domain_df['약어'].to_list() : 
        tmp_df = domain_df[domain_df['약어']== target_last_word]
        if len(tmp_df['데이터타입'].to_list()) != 1 :
            select_data_length = find_nearlist_val(sorted(tmp_df[tmp_df['데이터타입']==data_type]['데이터길이'].to_list()), data_length)
            if select_data_length != '' :     
                domain = tmp_df[(tmp_df['데이터타입']==data_type)&(tmp_df['데이터길이']==select_data_length)]['공통표준도메인명'].to_list()[0]
                return domain
            else : 
                return ''
        else :  
            select_data_length = find_nearlist_val(sorted(tmp_df['데이터길이'].to_list()), data_length)
            if select_data_length != '' :       
                domain = tmp_df[tmp_df['데이터길이']==select_data_length]['공통표준도메인명'].to_list()[0]
                return domain
            else :
                return ''                    
    else : 
        return ''
    


In [24]:
target_df = target_df.astype({  '한글용어명' : 'string',
                                '영문용어명' : 'string',
                                '분류어chk' : 'string',
                                '데이터 타입' : 'string',
                                '데이터 길이' : 'string'    })

target_df['데이터 길이']=target_df['데이터 길이'].apply(lambda x : x.replace(',','.'))
target_df = target_df.astype({'데이터 길이' : 'float'})

target_df

,한글용어명,영문용어명,분류어chk,데이터 타입,데이터 길이
217,사업소팩스번호,BIZP_FXNO,FXNO,VARCHAR2,15.0
218,지역전화번호,AREA_TELNO,TELNO,VARCHAR2,4.0
221,구매담당여부,BUY_TCHG_YN,YN,CHAR,1.0
222,공사담당여부,CONS_TCHG_YN,YN,CHAR,1.0
223,용역담당여부,SVL_TCHG_YN,YN,CHAR,1.0
224,간이공사담당여부,SMPLC_CONS_TCHG_YN,YN,CHAR,1.0
225,하도급담당여부,SBCN_TCHG_YN,YN,CHAR,1.0
228,최상위관리자여부,UPPOS_MNGR_YN,YN,CHAR,1.0
231,상태코드값,STS_CD_VAL,VAL,CHAR,1.0
238,리스트코드명,LIST_CD_NM,NM,VARCHAR2,10.0


In [68]:
target_df['도메인명']=target_df.apply(lambda row : select_domin(domain_df, row['분류어chk'], row['데이터 타입'], row['데이터 길이']), axis=1)
target_df['최대데이터길이']=target_df.apply(lambda row : select_max_col_length(col_maxlen_df, row['분류어chk'], row['데이터 타입']), axis=1)
target_df.reset_index(inplace=True, drop=True)
target_df

,level_0,index,한글용어명,영문용어명,분류어chk,데이터 타입,데이터 길이,도메인명,최대데이터길이
0,0,215,사업소명,BIZP_NM,NM,VARCHAR2,50.0,명V80,50.0
1,1,216,사업소주소,BIZP_ADDR,ADDR,VARCHAR2,200.0,주소V200,200.0
2,2,217,사업소팩스번호,BIZP_FXNO,FXNO,VARCHAR2,15.0,팩스번호V20,15.0
3,3,218,지역전화번호,AREA_TELNO,TELNO,VARCHAR2,4.0,전화번호V4,4.0
4,4,220,사번,EMPNO,EMPNO,VARCHAR2,8.0,,20.0
5,5,221,구매담당여부,BUY_TCHG_YN,YN,CHAR,1.0,여부C1,1.0
6,6,222,공사담당여부,CONS_TCHG_YN,YN,CHAR,1.0,여부C1,1.0
7,7,223,용역담당여부,SVL_TCHG_YN,YN,CHAR,1.0,여부C1,1.0
8,8,224,간이공사담당여부,SMPLC_CONS_TCHG_YN,YN,CHAR,1.0,여부C1,1.0
9,9,225,하도급담당여부,SBCN_TCHG_YN,YN,CHAR,1.0,여부C1,1.0


In [81]:
upload_df = pd.DataFrame(columns=['요청유형','용어명','용어영문명','도메인명','표준여부','용어설명','*최대데이터길이'])

upload_df['용어명']=target_df['한글용어명']
upload_df['용어영문명']=target_df['영문용어명']
upload_df['도메인명']=target_df['도메인명']
upload_df['용어설명']=target_df['한글용어명']
upload_df['*최대데이터길이']=target_df['최대데이터길이']
upload_df['요청유형']='신규'
upload_df

,요청유형,용어명,용어영문명,도메인명,표준여부,용어설명,*최대데이터길이
0,신규,사업소명,BIZP_NM,명V80,NaN,사업소명,50.0
1,신규,사업소주소,BIZP_ADDR,주소V200,NaN,사업소주소,200.0
2,신규,사업소팩스번호,BIZP_FXNO,팩스번호V20,NaN,사업소팩스번호,15.0
3,신규,지역전화번호,AREA_TELNO,전화번호V4,NaN,지역전화번호,4.0
4,신규,사번,EMPNO,,NaN,사번,20.0
5,신규,구매담당여부,BUY_TCHG_YN,여부C1,NaN,구매담당여부,1.0
6,신규,공사담당여부,CONS_TCHG_YN,여부C1,NaN,공사담당여부,1.0
7,신규,용역담당여부,SVL_TCHG_YN,여부C1,NaN,용역담당여부,1.0
8,신규,간이공사담당여부,SMPLC_CONS_TCHG_YN,여부C1,NaN,간이공사담당여부,1.0
9,신규,하도급담당여부,SBCN_TCHG_YN,여부C1,NaN,하도급담당여부,1.0


In [39]:
def select_max_col_length(col_maxlen_df, target_last_word, data_type) :
    if target_last_word in col_maxlen_df['분류어chk'].to_list() :
        if data_type in col_maxlen_df[col_maxlen_df['분류어chk']==target_last_word]['데이터 타입'].to_list():
            max_length = col_maxlen_df[(col_maxlen_df['분류어chk']==target_last_word)&(col_maxlen_df['데이터 타입']==data_type)]['데이터 길이'].to_list()[0]
            return max_length
        else :
            return ''
    else :
        return '' 

In [33]:
file_path = '/Users/jihyun/Desktop/upload_terms/asset/표준화 작업-71.계약이행실적증명발급_V0.97.xlsx'

wb = load_workbook(filename=file_path, data_only=True)
sheetname = wb.sheetnames[1]

df = pd.read_excel(file_path, sheet_name=sheetname)
print(f'read_excel_file : {file_path} | Sheet name : {sheetname}')
tmp_df = df.rename(columns=df.iloc[0])
extract_df = tmp_df.drop(tmp_df.index[0]) 
extract_df.fillna('', inplace=True)
extract_df= extract_df[(extract_df['분류어chk']!='CD')&(extract_df['미사용구분']=='')&(extract_df['분류어chk']!='')]
target_df = extract_df[['한글용어명','영문용어명','분류어chk','데이터 타입','데이터 길이']]


target_df = target_df.astype({  '한글용어명' : 'string',
                                '영문용어명' : 'string',
                                '분류어chk' : 'string',
                                '데이터 타입' : 'string',
                                '데이터 길이' : 'string'    })

target_df['데이터 길이']=target_df['데이터 길이'].apply(lambda x : x.replace(',','.'))
target_df = target_df.astype({'데이터 길이' : 'float'})



read_excel_file : /Users/jihyun/Desktop/upload_terms/asset/표준화 작업-71.계약이행실적증명발급_V0.97.xlsx | Sheet name : ★메타데이터


,한글용어명,영문용어명,분류어chk,데이터 타입,데이터 길이
215,사업소명,BIZP_NM,NM,VARCHAR2,50.0
216,사업소주소,BIZP_ADDR,ADDR,VARCHAR2,200.0
217,사업소팩스번호,BIZP_FXNO,FXNO,VARCHAR2,15.0
218,지역전화번호,AREA_TELNO,TELNO,VARCHAR2,4.0
220,사번,EMPNO,EMPNO,VARCHAR2,8.0
221,구매담당여부,BUY_TCHG_YN,YN,CHAR,1.0
222,공사담당여부,CONS_TCHG_YN,YN,CHAR,1.0
223,용역담당여부,SVL_TCHG_YN,YN,CHAR,1.0
224,간이공사담당여부,SMPLC_CONS_TCHG_YN,YN,CHAR,1.0
225,하도급담당여부,SBCN_TCHG_YN,YN,CHAR,1.0


In [40]:
col_maxlen_df=target_df.groupby(['분류어chk','데이터 타입'])['데이터 길이'].max().reset_index()

In [10]:
import glob
upload_file_sc = glob.glob('C:/Users/moonj/Desktop/남동발전표준화/자동화/upload_terms/asset/sample/*.xlsx')
upload_file = upload_file_sc[0].replace('sample','result')

target_file = glob.glob('C:/Users/moonj/Desktop/남동발전표준화/자동화/upload_terms/asset/*.xlsx')
# upload_file = upload_file.replace('용어등록 샘플.xlsx',target_file[0].split('')[-1])
# upload_file

target_file

['C:/Users/moonj/Desktop/남동발전표준화/자동화/upload_terms/asset\\표준화 작업-71.계약이행실적증명발급_V0.97.xlsx']

In [14]:

file_path = 'asset/표준화 작업-71.계약이행실적증명발급_V0.97.xlsx'

upload_file_sc = glob.glob('asset/sample/*.xlsx')

upload_file = upload_file_sc[0].replace('//','/')
upload_file = upload_file.replace('sample','result')
upload_file = upload_file.replace('용어등록 샘플.xlsx',file_path.split('/')[-1])


upload_file

'asset/result/SMETA_표준화 작업-71.계약이행실적증명발급_V0.97.xlsx'

In [57]:
import shutil

shutil.copy(upload_file_sc[0],upload_file)



'/Users/jihyun/Desktop/upload_terms/asset/result/SMETA_표준화 작업-71.계약이행실적증명발급_V0.97.xlsx'

In [83]:
print('Excel UPDATE progress start')
writer = pd.ExcelWriter(upload_file, engine='openpyxl', mode='a', if_sheet_exists='overlay')

upload_df.to_excel  (   writer,
                        sheet_name='용어목록',
                        header=False,
                        index=False, 
                        startrow=1,
                        startcol=1)

writer.close()

Excel UPDATE progress start


In [15]:
a= ['1','2','3','4','5']


for i in range(len(a)):
    test_str = '_'.join(a[i:])
    print(test_str)


1_2_3_4_5
2_3_4_5
3_4_5
4_5
5


In [50]:
a= ['1','2','3','4','5']
b= ['6','7','8','9','10']

for i,j in enumerate(zip(a , b)) :
    print(i)
    print(j)
    print(j[0])
    print(j[1])

0
('1', '6')
1
6
1
('2', '7')
2
7
2
('3', '8')
3
8
3
('4', '9')
4
9
4
('5', '10')
5
10


In [51]:
file_list = glob.glob('asset/표준화 작업*.xlsx')
dt_list = glob.glob('asset/data/*.xlsx')


In [52]:
file_list

['asset\\표준화 작업-71.계약이행실적증명발급_V0.97.xlsx']

In [53]:
dt_list

['asset/data\\01.ICT종합관리_용어도메인작업.xlsx']

In [54]:
for i,j in enumerate(zip(file_list, dt_list)):
        file_path = j[0].replace("\\","/")
        dt_path = j[1].replace("\\", "/")
        
        print(file_path)
        print(dt_path)

asset/표준화 작업-71.계약이행실적증명발급_V0.97.xlsx
asset/data/01.ICT종합관리_용어도메인작업.xlsx


In [56]:
df = pd.read_excel(dt_path)
df.fillna('',inplace=True)

df

,한글용어명,영문용어명,분류어chk,데이터 타입,데이터 길이
0,결재순서,APRV_SEQO,SEQO,NUMBER,
1,결재일련번호,APRV_SERNO,SERNO,NUMBER,
2,부재결재일시,ABSN_APRV_DT,DT,DATE,
3,결재자부서명,APRVR_DEPT_NM,NM,VARCHAR2,
4,결재URL주소,APRV_URL_ADDR,ADDR,VARCHAR2,
...,...,...,...,...,...
827,5ICT종합관리유형별탭코드,N5_ICTTMS_TP_TAP_CD,CD,,
828,ICT종합관리할일구분코드,ICTTMS_TODO_CL_CD,CD,,
829,ICT종합관리서비스신청구분코드,ICTTMS_SVC_APL_CL_CD,CD,,
830,ICT종합관리성능측정구분코드,ICTTMS_PRFM_MSR_CL_CD,CD,,


In [59]:
type(df[df['한글용어명']=='결재순서']['분류어chk'][0])


str